# Automatic Cell Detection with Tensorflow 

This is a study for identification of different cell types in biological tissue images. There are many types of cells present in a tissue. Some can be cancerous cells, immune cells while many others which look like a cell but actually not cell at all. This excercise uses Tensorflow to classify a valid cell from a non cell object from a given tissue image. 

I have collected some Immuno Histological Chemistry (IHC) stained images of brain tumor cells. I wrote an pre-processing algorithm to dentify the cell regions for a given stained image. In this project, I have developed an automatic cell detection framework using Tensorflow Object Detection API.


## Dataset description

The initial dataset consist of 110 svs wholeslide images of brain tumor cell. The cells, which are dark circular regions comes in variety of shapes and sizes and are stained differently. To bring all images in same color, I have normalized the color coordinates. 

<table> 
<tr>
<td colspan="3"> <img src="Images_Jupyter/whole_slide_image.png" alt="Drawing" style="width: 600px; height:300px"/> </td>
</tr>
<tr>
<td colspan="3"> <div align="center"> IHC stained whole slide image sample </div> </td>
</tr>
<tr>
<td> <img src="Images_Jupyter/image_patch1.png" alt="Drawing" style="width: 200px;"/> </td>
<td> <img src="Images_Jupyter/image_patch2.png" alt="Drawing" style="width: 200px;"/> </td>
<td> <img src="Images_Jupyter/image_patch3.png" alt="Drawing" style="width: 200px;"/> </td>
</tr>
<tr>
<td colspan="3"> <div align="center"> Extracted Image Patches </div> </td>  
</tr>
</table>

## Pre-processing

First, small patches are extracted from raw images in MATLAB. The corresponding code is attached here:

<img src="Images_Jupyter/patch_generate.png" >

I have generated semi-hand annotations for the cell regions in each patch. Implemented a crude algorithm for cell detection and then further refined by hand(by removing the false positive). The accuracy of this annotation is only ($~85\%$).

<img src="Images_Jupyter/matlab_BB_anno.png" >

The rsulting patches looks like: 
<table>
    <tr>
        <td> <img src="Images_Jupyter/train_image1_BB_anno.png" style="width: 200px;"> </td>
        <td> <img src="Images_Jupyter/train_image2_BB_anno.png" style="width: 200px;"> </td>
    </tr>
    <tr>
        <td colspan="2"> <div align="center"> Generated cell patches </div> </td>
    <tr>
</table>

In [ ]:
from xml.etree import ElementTree
from xml.dom import minidom
import json
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
from ElementTree_pretty import prettify, xml_write
import xml.etree.cElementTree as ET
import os
import glob
import pandas as pd
import io
import tensorflow as tf
import skimage
import numpy as np
from skimage import io, transform
import shutil

## Preparing Inputs: Dataset and annotations

I have used [bmidb0 cluster](http://bmidb.cs.stonybrook.edu/) GPU for current study and cloned the models folder into the repository root directory. I have installed the TensorFlow API by running the following commands from root directory. My working folder is ../tensorflow/models/research

Due to time constraint, I could have generated the ground truth annotations of 50 images, used for the  training/validation. The corresponding images are saved in ../research/images/train folder. Since, the bounding box annotations were saved in json file in MATLAB, I have written scripts to convert them into standard PASCAL VOC xml format from json. The xml annotation files are saved in ../research/annotations folder. After that, the xml files are converted to csv by runnning the script xml_to_csv.py

### Convert json to XML annotations 

In [3]:
## reference site to write xml file from json data: https://pymotw.com/2/xml/etree/ElementTree/create.html


'''ElementTree_pretty.py '''

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def xml_write(elem, filename):
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    with open(filename,'w') as write_file:
        write_file.write(reparsed.toprettyxml(indent="  "))

In [ ]:
'''createXmlFromJson.py''' 
'''This function will write the bounding box annotations along with other informations in xml 
files similar of PASCAL VOC format'''

Path = "/data/scratch/mnroy/DeepLearning/Project1/tensorflow/models/research/anno_json/"
filelist = os.listdir(Path)
for i in filelist:
    if i.endswith(".json"):  # You could also add "and i.startswith('f')
        with open(Path + i, 'r') as data_file:
            data = json.load(data_file)
            
    filename = "/data/scratch/mnroy/DeepLearning/Project1/tensorflow/models/research/annotations/" + 
    data['filename'].split('.')[0] + '.xml'

    xml = '''<?xml version="1.0" encoding="UTF-8"?>
    '''
    top = Element('annotation')
    child1 = SubElement(top, 'folder')
    child1.text = data['folder']
    child2 = SubElement(top, 'filename')
    child2.text = data['filename']

    child3 = SubElement(top, 'path')
    child3.text = "/data/scratch/mnroy/DeepLearning/Project1/tensorflow/models/research/images/train/" + 
    data['filename']

    source = SubElement(top, 'source')
    database = SubElement(source, 'database')
    database.text = "Unknown"
    size = SubElement(top, 'size')

    width = SubElement(size, 'width')
    width.text = str(data['size'][0])

    height = SubElement(size, 'height')
    height.text = str(data['size'][1])

    depth = SubElement(size, 'depth')
    depth.text = str(data['size'][2]) 

    child4 = SubElement(top, 'segmented')
    child4.text = str(0)

    #for k in range(len(data['bndbox'])):
    for bndbox,img_cls in zip(data['bndbox'],data['image_class']):
        obj = SubElement(top, 'object')

        name = SubElement(obj, 'name')
        if img_cls:
                    name.text = 'Cell'
        else:
                    name.text='Non-Cell'
 
        pose = SubElement(obj, 'pose')
        pose.text = 'Unspecified'
        truncated = SubElement(obj, 'truncated')
        truncated.text = str(0)
        difficult = SubElement(obj, 'difficult')
        difficult.text = str(0)
        bndbox = SubElement(obj, 'bndbox')
        xmin = SubElement(bndbox, 'xmin')
        xmin.text = str(bndbox[0])
        ymin = SubElement(bndbox, 'ymin')
        ymin.text = str(bndbox[1])
        xmax = SubElement(bndbox, 'xmax')
        xmax.text = str(bndbox[2])
        ymax = SubElement(bndbox, 'ymax')
        ymax.text = str(bndbox[3])

    xml_write(top, filename)


### Convert XML Labels to CSV 

In [ ]:
# Modified from source: https://github.comr/datitran/raccoon_dataset/blob/master/xml_to_csv.py

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def main():
    image_path = os.path.join(os.getcwd(), 'annotations')
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('data/cell_labels.csv', index=None)
    print('Successfully converted xml to csv.')


main()

### Label Maps 

The label map associated with the dataset defining a mapping from string class names to integer class Ids, are stored in ../research/data/cell_label_map.pbtxt. The label map of my dataset corersponds to two classes, Cell and Non-Cell. It is very important to start the label map from id 1, since the index 0 is a placeholder index.

In [ ]:
item {
  id: 1
  name: 'Cell'
}

item {
  id: 2
  name: 'Non-Cell'
}

In [ ]:
## split labels.ipynb : used to split the full labels into train and test labels. 
## Reference: https://github.com/datitran/raccoon_dataset/blob/master/split%20labels.ipynb

np.random.seed(1)
full_labels = pd.read_csv('data/cell_labels.csv')

full_labels.head()
grouped = full_labels.groupby('filename')
grouped.apply(lambda x: len(x)).value_counts()
full_labels.count()
total_data = full_labels['filename']

gb = full_labels.groupby('filename')
grouped_list = [gb.get_group(x) for x in gb.groups]
len(grouped_list)

train_index = np.random.choice(len(grouped_list), size=40, replace=False)
test_index = np.setdiff1d(list(range(50)), train_index)
len(train_index), len(test_index)

# take first 10 files
train = pd.concat([grouped_list[i] for i in train_index])
test = pd.concat([grouped_list[i] for i in test_index])
len(train), len(test)
train.to_csv('data/train_labels.csv', index=None)
test.to_csv('data/test_labels.csv', index=None)

### Create TF Record 

The Tensorflow Object Detection API expects data to be in the TFRecord format, so we'll now run the create_cell_tf_record script to convert from the raw  dataset into TFRecords.

In [ ]:
# source: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md

"""
Usage:
  # From tensorflow/models/research
  # Create train data:
  python create_Cell_tf_record.py --csv_input=data/train_labels.csv  --output_path=data/train.record
  # Create test data:
  python create_Cell_tf_record.py --csv_input=data/test_labels.csv  --output_path=data/test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
FLAGS = flags.FLAGS

# TO-DO replace this with label map
 def class_text_to_int(row_label):
    if row_label == 'Cell':
        return 1

    if row_label == 'Non-Cell':
        return 2 
    else:
        None
   
 def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


 def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_png = fid.read()
    encoded_png_io = io.BytesIO(encoded_png)
    image = Image.open(encoded_png_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'png'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        #print(class_text_to_int(row['class']))
        classes.append(class_text_to_int(row['class']))


    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_png),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


 def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(os.getcwd(), 'images/train')
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.app.run()

## Download Model for Transfer Learning

Training an object detector from scratch can take days, even when using multiple GPUs!. For my work, accuracy of Cell locations is more important than speed of training. Therefore, I have used Faster-RCNN-ResNet object detection model trained on the [COCO dataset](http://cocodataset.org/#home), and reused some of it's parameters to initialize my new model. I have downloaded COCO-pretrained Faster R-CNN with Resnet-101 model and unzipped the contents of the folder and copied the model.ckpt* files into data folder. 

## Configuring the Object Detection Training Pipeline 

I have configured the [config](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs) file for resnet101 and saved as ../resaerch/data/faster_rcnn_resnet101_Cell.config.
I have changed the num_classes to two and also set the path (PATH_TO_BE_CONFIGURED) for the model checkpoint, the train and test data files as well as the label map. The other configurations like the learning rate, batch size etc., I kept their default settings. I have used num_steps as 100000. I have used data_augmentation_options (i) random_vertical_flip{} and (ii) random_rotation90{} apart from their default one, random_horizontal_flip{}

At this point, I do have the training/validation datasets (including label map), COCO trained FasterRCNN finetune checkpoint and configuration file in the ../research/data folder, which should look like the following:

## Train Model

Since, I have used Transfer learning from the pre trained model, the training time is much less. I have used cluster GPU(node0) for training and CPU for evaluation at the same time. I have run the follwoing code for traning and evaluation in two different screens in the repository root directory. It is much easier to monitor the process of the training and evaluation jobs by running Tensorboard on the cluster. Once the loss drops to a consistant level for a while, TensorFlow training can be stopped by pressing ctrl+c.
To start training and evaluation, the following commands are executed from the tensorflow/models/research/  directory

### Monitoring Progress with Tensorboard 

I have started the TensorBoard in bmidb0 cluster to monitor the total loss and other evaluation parameters, by running the below command from ../tensflow/models directory

<img src= "Images_Jupyter/tensor_board.png" width="1000">

## Analysis of result 

Here are the results from my training and evaluation jobs. In total, I ran it over few hours/60k steps with a batch size of 1 (due to GPU memory limitations), but I already achieved reasonable results after about 30k steps.
This is how the total loss evolved. Total loss decreased quite fast due to the pre-trained model.

<table>
    <tr>
        <td> <img src="Images_Jupyter/Total_Loss.png" width="800"> </td>
    </tr>
    <tr>
        <td> <div align="center"> Total Loss saturating at ~30k steps </div> </td>
    </tr>
</table>

The mAP (mean average precision) at 0.5IoU for Cell, Non-Cell and total are shown below. Since the Cell regions are main focus of this study, the mAP value for Cell 0.7590 at 38k step is quite good to achieve. The Non-Cell regions are quite a bit unclear from the images, therefore the mAP estimate is quite low resulting in lower value for total mAP estimate.

<table>
    <tr>
        <td> <img src="Images_Jupyter/tensorboard_mAP.png" width="800"> </td>
    </tr>
    <tr>
        <td> <div align="center"> mAP estimates </div> </td>
    </tr>
</table>

Example for the evaluation of few images while training the model:

<table>
  <tr>
    <td> <img src="Images_Jupyter/Initial_eval_step0.png" alt="Drawing" style="width: 500px;"/> </td>
    <td> <img src="Images_Jupyter/Images_evalEnd.png" alt="Drawing" style="width: 500px;"/> </td>
  </tr>
  <tr>
      <td> <div align="center"> Initial evaluation at step 0 of the training model. </div> </td>
      <td> <div align="center"> Evaluation at end of training. </div> </td>
  </tr>
</table>

### Object Detection Utilities copied to Root Directory 

I have copied some of the utilities from the Object Detection folder into the root directory.

## Exporting the trained model for inference 

After the model has been trained, it is required to export it to a Tensorflow graph proto. First I have created a directory exported_graphs in ../research folder where output_inference_graph.pb will be saved. Then, I have identified a candidate checkpoint from the file stored at ../research/train folder followed by executing the below command from tensorflow/models/research/:

## Test Model 

In [ ]:
# Modified From API
# https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

from utils import label_map_util
from utils import visualization_utils as vis_util

# This is needed to display the images.
#%matplotlib inline

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'exported_graphs/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'data/cell_label_map.pbtxt'

NUM_CLASSES = 2

PATH_TO_TEST_IMAGES_DIR = 'images/test'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.png'.format(i)) for i in range(1, 10) ]
IMAGE_SIZE = (8, 8)

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


with detection_graph.as_default():
      with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=1)
            plt.figure(figsize=IMAGE_SIZE)
            fig = plt.imshow(image_np)
            fig.set_cmap('hot')
            plt.axis('off')
            fig.axes.get_xaxis().set_visible(False)
            fig.axes.get_yaxis().set_visible(False)
            #plt.show()

            filename = 'test_result/' + image_path.split('/')[-1]
            #filename =  image_path.split('/')[-1]
            plt.savefig(filename, bbox_inches='tight', pad_inches = 0)

### Test Result Images 

<table>
<tr>
    <td> <img src="Images_Jupyter/image8.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="Images_Jupyter/image7.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="Images_Jupyter/image6.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="Images_Jupyter/image3.png" alt="Drawing" style="width: 250px;"/> </td>
</tr>
<tr>
    <td colspan="4"> <div align="center"> Test Images </div> </td>
</tr>
</table>

## Conclusion

From the above study, we can infer that Faster rcnn resnet model is performing quite well on unseen images and able to detect most of the Cell locations. Although, its performance degrades when numerous number of Cells are present in the image patch. This is due to very limited number of training data and the ground truth accuracy of bounding box annotations is also quite less. This is a research project, I will be pursuing further on this topic and prepare it for publication in near future.